<a href="https://colab.research.google.com/github/pmnerfed/capstone_option_trading/blob/main/Review/Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Long/Short Term Trading Strategies

In [11]:
# prompt: import and analyse AAPL stock option data

import pandas as pd
from datetime import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import math
from datetime import datetime


## Gathering the data

In [2]:
# prompt: import and analyse AAPL stock option data

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Load AAPL stock option data
ticker = 'AAPL'
expiry = '2023-03-17'
strike = 140

# Get option chain data
option_chain = yf.Ticker(ticker).option_chain()

# Print option chain data
print(option_chain)

# Plot the option chain data
option_chain.plot_option_chains()
plt.show()


Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice  bid  \
0   AAPL240119C00050000 2024-01-18 16:58:29+00:00    50.0     138.47  0.0   
1   AAPL240119C00055000 2024-01-18 16:42:45+00:00    55.0     133.35  0.0   
2   AAPL240119C00060000 2024-01-18 15:59:18+00:00    60.0     128.00  0.0   
3   AAPL240119C00065000 2024-01-17 19:26:11+00:00    65.0     117.50  0.0   
4   AAPL240119C00070000 2024-01-18 16:40:32+00:00    70.0     118.38  0.0   
..                  ...                       ...     ...        ...  ...   
63  AAPL240119C00285000 2024-01-08 18:32:07+00:00   285.0       0.01  0.0   
64  AAPL240119C00290000 2024-01-09 14:30:00+00:00   290.0       0.01  0.0   
65  AAPL240119C00300000 2024-01-18 16:01:33+00:00   300.0       0.01  0.0   
66  AAPL240119C00310000 2024-01-10 14:30:00+00:00   310.0       0.01  0.0   
67  AAPL240119C00320000 2024-01-16 20:54:28+00:00   320.0       0.01  0.0   

    ask  change  percentChange  volume  openInterest  implied

AttributeError: 'Options' object has no attribute 'plot_option_chains'

In [ ]:
# prompt: import and analyse AAPL stock option data

import pandas as pd
import numpy as np
from datetime import datetime

# Import AAPL stock option data
df = pd.read_csv('AAPL_stock_options.csv')

# Convert the expiration date column to a datetime object
df['expiration_date'] = pd.to_datetime(df['expiration_date'])

# Calculate the strike price
df['strike_price'] = df['strike_price'].astype(int)

# Calculate the option type (call or put)
df['option_type'] = df['option_type'].str.lower()

# Calculate the option premium
df['option_premium'] = df['option_premium'].astype(float)

# Calculate the bid price
df['bid_price'] = df['bid_price'].astype(float)

# Calculate the ask price
df['ask_price'] = df['ask_price'].astype(float)

# Calculate the last traded price
df['last_traded_price'] = df['last_traded_price'].astype(float)

# Calculate the volume
df['volume'] = df['volume'].astype(int)

# Calculate the open interest
df['open_interest'] = df['open_interest'].astype(int)

# Calculate the implied volatility
df['implied_volatility'] = df['implied_volatility'].astype(float)

# Calculate the time to expiration
df['time_to_expiration'] = df['expiration_date'] - datetime.now()

# Calculate the intrinsic value
df['intrinsic_value'] = np.where(df['option_type'] == 'call', df['last_traded_price'] - df['strike_price'], df['strike_price'] - df['last_traded_price'])

# Calculate the extrinsic value
df['extrinsic_value'] = df['option_premium'] - df['intrinsic_value']

# Calculate the delta
df['delta'] = np.where(df['option_type'] == 'call', np.where(df['last_traded_price'] > df['strike_price'], 1, 0), np.where(df['last_traded_price'] < df['strike_price'], -1, 0))

# Calculate the gamma
df['gamma'] = np.where(df['option_type'] == 'call', np.where(df['last_traded_price'] > df['strike_price'], 0.5, -0.5), np.where(df['last_traded_price'] < df['strike_price'], -0.5, 0.5))

# Calculate the vega
df['vega'] = np.where(df['option_type'] == 'call', 0.01 * df['last_traded_price'] * np.sqrt(df['time_to_expiration'].dt.days / 365), -0.01 * df['last_traded_price'] * np.sqrt(df['time_to_expiration'].dt.days / 365))

# Calculate the theta
df['theta'] = np.where(df['option_type'] == 'call', -0.01 * df['last_traded_price'] * df['implied_volatility'] * np.sqrt(df['time_to_expiration'].dt.days / 365), 0.01 * df['last_traded_price'] * df['implied_volatility'] * np.sqrt(df['time_to_expiration'].dt.days / 365))

# Calculate the rho
df['rho'] = np.where(df['option_type'] == 'call', 0.01 * df['strike_price'] * df['time_to_expiration'].dt.days / 365, -0.01 * df['strike_price'] * df['time_to_expiration'].dt.days / 365)

# Print the head of the dataframe
print(df.head())

## Exploratory Analysis

### Cleanup

## Analysis of existing non-deep learning strategies

### Defining Strategy models

In [18]:
import os
from pathlib import Path

Path("strategies").mkdir(parents=True, exist_ok=True)

#### Simple Moving Average

In [29]:
!pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=5674f8f7611581a26425320a19c7a7682332c22a9a96add3312eceea0f9d8f3b
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [33]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import GOOG



In [38]:
%%writefile strategies/sma.py
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA

class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

Overwriting strategies/sma.py


### Defining scoring and backtesting methodology

In [40]:
from strategies.sma import SmaCross

bt = Backtest(GOOG, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()


ImportError: cannot import name 'SmaCross' from 'strategies.sma' (/content/strategies/sma.py)

### Comparing existing strategies

## Analysisng existing Deep learning Strategies

### Defining strategy models

### Comparing with existing models

### Analysis of strategies and their pros and cons

## Proposing our strategy

> Explain the strategy in words

### Defining the strategy

### Comparing strategies with various methods

## Conclusion